<a href="https://colab.research.google.com/github/Simon-creator-jara/Inteligencia_artificial/blob/main/Mapa_Belgica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pydot
from IPython.display import Image, display
import queue
import numpy as np

In [ ]:
class Node ():
  def __init__(self, state,value,operators,operator=None, parent=None,objective=None):
    self.state= state
    self.value = value
    self.children = []
    self.parent=parent
    self.operator=operator
    self.objective=objective
    self.level=0
    self.operators=operators

    
  def add_child(self, value, state, operator):
    node=type(self)(value=value, state=state, operator=operator,parent=self,operators=self.operators)
    node.level=node.parent.level+1
    self.children.append(node)
    return node
  
  def add_node_child(self, node):
    node.level=node.parent.level+1
    self.children.append(node)    
    return node

  #Devuelve todos los estados según los operadores aplicados
  def getchildrens(self):
    return [
        self.getState(i) 
          if not self.repeatStatePath(self.getState(i)) 
            else None for i, op in enumerate(self.operators)]
    
  def getState(self, index):
    pass
  

  def __eq__(self, other):
    return self.state == other.state  

  def __gt__(self, other):
    return self.state > other.state  

  def __lt__(self, other):
    return self.state < other.state 
  
  def repeatStatePath(self, state):
      n=self
      while n is not None and n.state!=state:
          n=n.parent
      return n is not None
    
  def pathObjective(self):
      n=self
      result=[]
      while n is not None:
          result.append(n)
          n=n.parent
      return result
  
  def heuristic(self):
    return 0
  

In [ ]:
class Tree ():
  def __init__(self, root ,operators):
    self.root=root
    self.operators=operators

  def printPath(self,n):
    stack=n.pathObjective()
    path=stack.copy()
    while len(stack)!=0:
        node=stack.pop()
        if node.operator is not None:
            print(f'operador:  {self.operators[node.operator]} \t estado: {node.state}')
        else:
            print(f' {node.state}')
    return path

  def reinitRoot(self):
    self.root.operator=None
    self.root.parent=None
    self.root.objective=None
    self.root.children = []
    self.root.level=0

  ## Primero a lo ancho
  def breadthFirst(self,endState):
    self.reinitRoot()
    pq=queue.Queue()
    pq.put(self.root)
    while not pq.empty():
      node=pq.get()
      children=node.getchildrens()
      for i,child in enumerate(children):
        if child is not None:
          newChild=node.add_child(value=node.value+'-'+str(i), state=child, operator=i)
          pq.put(newChild)
          if endState==child:
            return newChild

  ## Primero en profundidad
  def dephFirst(self, endState):
    self.reinitRoot()
    pq=[]
    pq.append(self.root)
    while len(pq)>0:
      node=pq.pop()
      if (node.parent is not None):
        node.parent.add_node_child(node)
      children=node.getchildrens()
      temp=[]
      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node, operators=node.operators)
          temp.append(newChild)
          if endState==child:
            node.add_node_child(newChild)
            return newChild
      #Adicionar los hijos en forma inversa para que salga primero el primero que se adicionó
      temp.reverse()
      for e in temp:
        pq.append(e)
  
  ## Costo uniforme
  def costUniform(self,endState):
    self.reinitRoot()
    pq=queue.PriorityQueue()
    pq.put((self.root.cost(),self.root))
    while not pq.empty():
      node=pq.get()[1]
      children=node.getchildrens()
      for i,child in enumerate(children):
        if child is not None:
          newChild=node.add_child(value=node.value+'-'+str(i), 
                                  state=child, operator=i)
          pq.put((newChild.cost(),newChild))
          if endState==child:
            return newChild

  ## Primero el mejor 
  def bestFirst(self,endState):
    lista=[]
    

    self.reinitRoot()
    pq=queue.PriorityQueue()
    pq.put((self.root.heuristic(),self.root))
    while not pq.empty():
      node=pq.get()[1]
      #lista.append(node.state)
      if node.colores[node.state]==0:
        count=0
        uti=set()
        colors=[1,2,3,4,5]
        lista_adj=node.adj_colors(node.state)
        
        for c,co in lista_adj:
          if node.colores[c]==0:
            count=count+1
            print(count)
          else:
            uti.add(node.colores[c])



        if count==len(lista_adj):
          node.colores[node.state]=1
          lista.append(node.state)
        else:
          for co in colors:
            if co not in list(uti):
              node.colores[node.state]=co
              lista.append(node.state)
              break
        print(f"colores:{node.colores}")
        print(f"lista_adj:{lista_adj}" )
      #print(list_adj )
      print(f"La lista es:{lista}")
      print(f"entra a bestFirst node: {node}")
      print(f"entra a bestFirst: {node.state}")
      children=node.getchildrens()
      print(f"entra a bestFirst: {children}")
      for i,child in enumerate(children):
        if child is not None:
          newChild=node.add_child(value=node.value+'-'+str(i), 
                                  state=child, operator=i)
          pq.put((newChild.heuristic(),newChild))
          
          if endState==(len(lista)==11):
            return lista

  ## A*
  def Aasterisk(self,endState):
    self.reinitRoot()
    pq=queue.PriorityQueue()
    pq.put((self.root.f(),self.root))
    while not pq.empty():
      node=pq.get()[1]
      children=node.getchildrens()
      for i,child in enumerate(children):
        if child is not None:
          newChild=node.add_child(value=node.value+'-'+str(i), 
                                  state=child, operator=i)
          pq.put((newChild.f(),newChild))
          if endState==child:
            return newChild
  
  ## Método para dibujar el árbol        
  def draw(self,path):
    graph = pydot.Dot(graph_type='graph')
    nodeGraph=pydot.Node(str(self.root.state)+"-"+str(0),
                          label=str(self.root.state),shape ="circle", 
                          style="filled", fillcolor="red")
    graph.add_node(nodeGraph)
    path.pop()
    return self.drawTreeRec(self.root,nodeGraph,graph,0,path.pop(),path)
  ## Método recursivo para dibujar el árbol  
  def drawTreeRec(self,r,rootGraph,graph,i,topPath,path):
    if r is not None:
      children=r.children
      for j,child in enumerate(children):
        i=i+1
        color="white"
        if topPath.value==child.value:
          if len(path)>0:topPath=path.pop()
          color='red'
        c=pydot.Node(child.value,label=str(child.state)+r"\n"+r"\n"+"f="+str(child.f()), 
                      shape ="circle", style="filled", 
                      fillcolor=color)
        graph.add_node(c)
        graph.add_edge(pydot.Edge(rootGraph, c, 
                                  label=str(child.operator)+'('+str(child.cost())+')'))
        graph=self.drawTreeRec(child,c,graph,i,topPath,path)  # recursive call
      return graph
    else:
      return graph

In [ ]:
class RouteMap(Node):
  #Definiendo mapa como una variable static
  mapAdj=None
  #Definiendo la distancia aérea como una variable static
  colores=None
  
  def __init__(self, mapAdj=None, colores=None,**kwargs):
    super(RouteMap, self).__init__(**kwargs)
    if mapAdj is not None:
        RouteMap.mapAdj = mapAdj
    if colores is not None:
        RouteMap.colores=colores
    self.adj=self.adj()

  
  def adj(self):
    state=self.state
    keys=[key for key in RouteMap.mapAdj]
    adj=[]
    if state in keys:
      adj=[(key,RouteMap.mapAdj[state][key]) for key in RouteMap.mapAdj[state]]
    cities=[key for key in RouteMap.mapAdj]
    for c in cities:
        k=[k for k in RouteMap.mapAdj[c]]
        if state in k:
          adj.append((c,RouteMap.mapAdj[c][state]))
    result=[a if not self.repeatStatePath(a) else None for a in adj]
    return result

  def adj_colors(self,ciudad):
    state=ciudad
    keys=[key for key in RouteMap.mapAdj]
    adj=[]
    if state in keys:
      adj=[(key,RouteMap.mapAdj[state][key]) for key in RouteMap.mapAdj[state]]
    cities=[key for key in RouteMap.mapAdj]
    for c in cities:
        k=[k for k in RouteMap.mapAdj[c]]
        if state in k:
          adj.append((c,RouteMap.mapAdj[c][state]))
    
    return adj
          
  def getState(self, index): 
    if len(self.adj)>0:
        for city,d in self.adj:
            if (self.operators[index]==city):
                return self.operators[index]
    return None
  
  def heuristic(self):
    count=0
    uti=[]
    colors=[1,2,3,4,5]
    print(f"Entra a heuristica:{self.state}")
    list_adj=self.adj_colors(self.state)
    print(f"lista_adj_heuristica:{list_adj}")
    for c in list_adj:
      count=count+1
    print(count)
    return count

In [ ]:
    if RouteMap.colores[self.state]==0:
      list_adj=self.adj_colors(self.state)
      print(list_adj)
      if len(list_adj) !=0:
        for c,co in list_adj:
          if RouteMap.colores[c] ==0:
            count+=1
          else:
            uti.append(RouteMap.colores[c])

      if count==len(list_adj):
        RouteMap.colores[c]=1
      else:
        for i in colors:
          if i not in list(uti):

            RouteMap.colores[c]=i
            break
      print(RouteMap.colores)
    print(list_adj )

In [ ]:
belgica=dict(
    Wvland=dict(Ovland = 0, Hainaut = 0),
    Ovland=dict(Antwerpen = 0, Vlaams = 0, Hainaut = 0),
    Antwerpen=dict(Limburg = 0, Vlaams = 0),
    Vlaams=dict(Bru = 0, Limburg = 0, Brabant = 0, Hainaut = 0, Liege = 0),
    Brabant=dict(Hainaut = 0, Liege = 0, Namur = 0),
    Limburg=dict(Liege = 0),
    Namur=dict(Hainaut = 0, Liege = 0, Luxembourg = 0),
    Liege=dict(Luxembourg = 0))

In [ ]:
colores=dict(
    Wvland = 0,
    Ovland = 0,
    Antwerpen = 0,
    Limburg = 0,
    Vlaams = 0,
    Bru = 0,
    Hainaut = 0,
    Brabant = 0,
    Namur = 0, 
    Liege = 0,
    Luxembourg = 0
    )

In [ ]:
initState= 'Vlaams'
endState = True

In [ ]:
operators=[op for op in colores]
operators.sort()
print(operators)

['Antwerpen', 'Brabant', 'Bru', 'Hainaut', 'Liege', 'Limburg', 'Luxembourg', 'Namur', 'Ovland', 'Vlaams', 'Wvland']


In [ ]:
nodeArad=RouteMap(mapAdj=belgica, colores=colores,value="inicio",state=initState, operators= operators)
routeAradBucharest=Tree(nodeArad,operators)
objective=routeAradBucharest.bestFirst(endState)

Entra a heuristica:Vlaams
lista_adj_heuristica:[('Bru', 0), ('Limburg', 0), ('Brabant', 0), ('Hainaut', 0), ('Liege', 0), ('Ovland', 0), ('Antwerpen', 0)]
7
1
2
3
4
5
6
7
colores:{'Wvland': 0, 'Ovland': 0, 'Antwerpen': 0, 'Limburg': 0, 'Vlaams': 1, 'Bru': 0, 'Hainaut': 0, 'Brabant': 0, 'Namur': 0, 'Liege': 0, 'Luxembourg': 0}
lista_adj:[('Bru', 0), ('Limburg', 0), ('Brabant', 0), ('Hainaut', 0), ('Liege', 0), ('Ovland', 0), ('Antwerpen', 0)]
La lista es:['Vlaams']
entra a bestFirst node: <__main__.RouteMap object at 0x7f990421af10>
entra a bestFirst: Vlaams
entra a bestFirst: ['Antwerpen', 'Brabant', 'Bru', 'Hainaut', 'Liege', 'Limburg', None, None, 'Ovland', None, None]
Entra a heuristica:Antwerpen
lista_adj_heuristica:[('Limburg', 0), ('Vlaams', 0), ('Ovland', 0)]
3
Entra a heuristica:Brabant
lista_adj_heuristica:[('Hainaut', 0), ('Liege', 0), ('Namur', 0), ('Vlaams', 0)]
4
Entra a heuristica:Bru
lista_adj_heuristica:[('Vlaams', 0)]
1
Entra a heuristica:Hainaut
lista_adj_heuristica:[

In [ ]:
state='Luxembourg'
def adj(state):
  keys=[key for key in belgica]
  adj=[]
  if state in keys:
    adj=[(key,belgica[state][key]) for key in belgica[state]]
  cities=[key for key in belgica]
  for c in cities:
    k=[k for k in belgica[c]]
    if state in k:
      adj.append((c,belgica[c][state]))
  return adj

In [ ]:
w=adj(state)

In [ ]:
w

[('Namur', 0), ('Liege', 0)]

In [ ]:
def borrar(l):
  return l.pop()

In [ ]:
colors=[1,2,3,4,5]
ope=operators.copy()
for c in operators:
  count=0
  uti=[]
  if colores[c]==0:
    list_adj=adj(c)
    print(c)
    print(list_adj)
    if len(list_adj) !=0:
      for ci,co in list_adj:
        if colores[ci] ==0:
          count+=1
        else:
          uti.append(colores[ci])
      uti=set(uti)
      print(uti)
      print(count)


      if count==len(list_adj):
        print(c)
        colores[c]=1
      else:
        for i in colors:
          if i not in list(uti):
            colores[c]=i
            break
      
  



Antwerpen
[('Limburg', 0), ('Vlaams', 0), ('Ovland', 0)]
set()
3
Antwerpen
Brabant
[('Hainaut', 0), ('Liege', 0), ('Namur', 0), ('Vlaams', 0)]
set()
4
Brabant
Bru
[('Vlaams', 0)]
set()
1
Bru
Hainaut
[('Wvland', 0), ('Ovland', 0), ('Vlaams', 0), ('Brabant', 0), ('Namur', 0)]
{1}
4
Liege
[('Luxembourg', 0), ('Vlaams', 0), ('Brabant', 0), ('Limburg', 0), ('Namur', 0)]
{1}
4
Limburg
[('Liege', 0), ('Antwerpen', 0), ('Vlaams', 0)]
{1, 2}
1
Luxembourg
[('Namur', 0), ('Liege', 0)]
{2}
1
Namur
[('Hainaut', 0), ('Liege', 0), ('Luxembourg', 0), ('Brabant', 0)]
{1, 2}
0
Ovland
[('Antwerpen', 0), ('Vlaams', 0), ('Hainaut', 0), ('Wvland', 0)]
{1, 2}
2
Vlaams
[('Bru', 0), ('Limburg', 0), ('Brabant', 0), ('Hainaut', 0), ('Liege', 0), ('Ovland', 0), ('Antwerpen', 0)]
{1, 2, 3}
0
Wvland
[('Ovland', 0), ('Hainaut', 0)]
{2, 3}
0


In [ ]:
colores

{'Antwerpen': 1,
 'Brabant': 1,
 'Bru': 1,
 'Hainaut': 2,
 'Liege': 2,
 'Limburg': 3,
 'Luxembourg': 1,
 'Namur': 3,
 'Ovland': 3,
 'Vlaams': 4,
 'Wvland': 1}

In [ ]:
count=0
uti=[]
colors=[1,2,3,4,5]
if colores['Wvland']==0:
    list_adj=adj('Wvland')
    print(list_adj)
    if len(list_adj) !=0:
      for c,co in list_adj:
        if colores[c] ==0:
          count+=1
        else:
          uti.append(colores[c])
print(count)
uti=set(uti)
print(uti)
if count==len(list_adj):
  colores['Wvland']=1
else:
  for i in colors:
    if i not in list(uti):
      print(i)
      colores['Wvland']=i
      break

[('Ovland', 0), ('Hainaut', 0)]
0
{2, 3}
1


In [ ]:
colores

{'Antwerpen': 1,
 'Brabant': 1,
 'Bru': 1,
 'Hainaut': 2,
 'Liege': 2,
 'Limburg': 3,
 'Luxembourg': 1,
 'Namur': 3,
 'Ovland': 3,
 'Vlaams': 4,
 'Wvland': 1}

In [ ]:
for c in operators:
  print(c)

Antwerpen
Brabant
Bru
Hainaut
Liege
Limburg
Luxembourg
Namur
Ovland
Vlaams
Wvland
